<a href="https://colab.research.google.com/github/IgorB99/Pokemon_Battle/blob/master/Pokemon_Colab_Color.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
class Pokemon():
  def __init__(self, name, hp, attack, defense, special_attack, special_defense, speed):
    self.name = name
    self.hp = hp
    self.attack = attack
    self.defense = defense
    self.special_attack = special_attack
    self.special_defense = special_defense
    self.speed = speed


In [58]:
import requests

base_api_url = 'https://pokeapi.co/api/v2/pokemon?limit=151'
dados_brutos = requests.get(base_api_url)

def quem_e_esse_pokemon(pokemon):
  if dados_brutos.status_code >= 200 and dados_brutos.status_code < 300:
    for informacao in dados_brutos.json()["results"]:
      if informacao["name"] == pokemon:
        dados_brutos_pokemon = requests.get(informacao["url"])
        if dados_brutos_pokemon.status_code >= 200 and dados_brutos_pokemon.status_code < 300:
          pokemon_data = dados_brutos_pokemon.json()
          pokemon = Pokemon(name = pokemon_data["name"],
                            hp = pokemon_data["stats"][0]["base_stat"],
                            attack = pokemon_data["stats"][1]["base_stat"],
                            defense = pokemon_data["stats"][2]["base_stat"],
                            special_attack = pokemon_data["stats"][3]["base_stat"],
                            special_defense = pokemon_data["stats"][4]["base_stat"],
                            speed = pokemon_data["stats"][5]["base_stat"])
  return pokemon

def pokemon_acao(acao, n):
  if n == 1:
    if acao == 1:
      dano = ((pokemon1.attack * 2) / pokemon2.defense) * 10
    else:
      dano = ((pokemon1.special_attack * 2) / pokemon2.special_defense) * 10
  else:
    if acao == 1:
      dano = ((pokemon2.attack * 2) / pokemon1.defense) * 10
    else:
      dano = ((pokemon2.special_attack * 2) / pokemon1.special_defense) * 10
  return dano

pokemon1 = input("Choose your pokemon: ")
pokemon2 = input("Choose your opponent pokemon: ")

pokemon1 = quem_e_esse_pokemon(pokemon1)
pokemon2 = quem_e_esse_pokemon(pokemon2)

while pokemon1.hp > 0 and pokemon2.hp > 0:
  pokemon1_acao = int(input(pokemon1.name + " use attack or special_attack (1/2): "))
  pokemon2_acao = int(input(pokemon2.name + " use attack or special_attack (1/2): "))

  if pokemon1.speed > pokemon2.speed:
    pokemon2.hp -= pokemon_acao(pokemon1_acao, 1)
    if pokemon2.hp > 0:
      pokemon1.hp -= pokemon_acao(pokemon2_acao, 2)
    print("{}'s hp: {:.2f}, {}'s hp: {:.2f}".format(pokemon1.name, pokemon1.hp, pokemon2.name, pokemon2.hp))
  else:
    pokemon1.hp -= pokemon_acao(pokemon2_acao, 2)
    if pokemon1.hp > 0:
      pokemon2.hp -= pokemon_acao(pokemon1_acao, 1)
    print("{}'s hp: {:.2f}, {}'s hp: {:.2f}".format(pokemon1.name, pokemon1.hp, pokemon2.name, pokemon2.hp))

if pokemon1.hp > pokemon2.hp:
  print(pokemon1.name + " win")
else:
  print(pokemon2.name + " win")

Choose your pokemon: mew
Choose your opponent pokemon: mewtwo
mew use attack or special_attack (1/2): 1
mewtwo use attack or special_attack (1/2): 1
mew's hp: 78.00, mewtwo's hp: 83.78
mew use attack or special_attack (1/2): 2
mewtwo use attack or special_attack (1/2): 2
mew's hp: 47.20, mewtwo's hp: 61.56
mew use attack or special_attack (1/2): 1
mewtwo use attack or special_attack (1/2): 2
mew's hp: 16.40, mewtwo's hp: 39.33
mew use attack or special_attack (1/2): 2
mewtwo use attack or special_attack (1/2): 1
mew's hp: -5.60, mewtwo's hp: 39.33
mewtwo win
